In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
import cv2
from preprocess import preprocessing
#from read_mias import *
import config
from object_detector import ObjectDetector
from custom_tensor_dataset import CustomTensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CrossEntropyLoss
from torch.nn import MSELoss
from torch.optim import Adam
from torchvision.models import resnet50
from sklearn.model_selection import train_test_split
from imutils import paths
import pickle
import torch
import torchvision
from torchvision import transforms, datasets, models
import torch
from torchvision.models.detection.retinanet import RetinaNet, RetinaNet_ResNet50_FPN_V2_Weights
import albumentations as A
from albumentations.pytorch import ToTensorV2
from custom_tensor_dataset import CustomTensorDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights
import random


https://debuggercafe.com/train-pytorch-retinanet-on-custom-dataset/#download-code



In [2]:
dir = os.getcwd()
info = pd.read_csv(dir +'/mias_info/labels.txt', sep = ' ', header = None)
info.columns = ['refno', 'tissue', 'class', 'severity','x','y','r']
info['severity'] = info['severity'].fillna('N')
info['image_id'] = info.refno.str.split('b', expand = True)[1].astype(int) -1
info.head()

,refno,tissue,class,severity,x,y,r,image_id
0,mdb001,G,CIRC,B,535,425,197.0,0
1,mdb002,G,CIRC,B,522,280,69.0,1
2,mdb003,D,NORM,N,NaN,NaN,NaN,2
3,mdb004,D,NORM,N,NaN,NaN,NaN,3
4,mdb005,F,CIRC,B,477,133,30.0,4


In [3]:
bm = info[info['severity'] != 'N']['refno']
notnormal = bm.tolist()

for i, name in enumerate(notnormal):
    notnormal[i] = name + '.pgm'

In [3]:
data_transform = A.Compose([
	A.Resize(512, 512),
	A.HorizontalFlip(p=0.5),
	A.Rotate(limit=15, p=1.0),
	A.RandomBrightnessContrast(p=0.5),
	ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


In [4]:
from custom_tensor_dataset import list_files_walk
from retinaNetDataset import customDataset


all_imgs = list_files_walk('mias_data/')

train, test = train_test_split(notnormal, test_size = 0.2, random_state = 42)

train_dataset = customDataset(imgs = train)
test_dataset = customDataset(imgs = test)

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = True,
                                          collate_fn = collate_fn)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = True,
                                          collate_fn = collate_fn)

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
for imgs, annotations in train_dataloader:
    imgs = list(img.to(device) for img in imgs)
    print(imgs[0].dtype)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

cpu


AttributeError: 'list' object has no attribute 'numpy'

In [ ]:
from retina_model import create_model
import config_retina

model = create_model(config_retina.NUM_CLASSES)

print(model)


In [7]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights, pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

# 2 features because using classifying into 3 severities: 
#  benign, malignant

model = get_model_instance_segmentation(2)

In [8]:
num_epochs = config.NUM_EPOCHS
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(train_dataloader)

train_losses = []  # Store epoch losses
val_loss = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for imgs, annotations in train_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 

        epoch_loss += losses.item()  # Make sure to use .item() to extract scalar from tensor
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    train_losses.append(epoch_loss)

Epoch 1/20, Loss: 11.61277368851006
Epoch 2/20, Loss: 4.299155537970364
Epoch 3/20, Loss: 4.442819310352206
Epoch 4/20, Loss: 3.8915538899600506
Epoch 5/20, Loss: 4.47911805473268
Epoch 6/20, Loss: 4.7197097688913345
Epoch 7/20, Loss: 4.171210697386414
Epoch 8/20, Loss: 8.852135960943997


: 

In [4]:
from custom_tensor_dataset import list_files_walk
from retinaNetDataset import customDataset


all_imgs = list_files_walk('mias_data/')

train, test = train_test_split(all_imgs, test_size = 0.2, random_state = 42)

train_dataset = customDataset(imgs = train, transforms = data_transform)
test_dataset = customDataset(imgs = test, transforms = data_transform)

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = True,
                                          collate_fn = collate_fn)

test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = config.BATCH_SIZE,
                                          shuffle = True,
                                          collate_fn = collate_fn)

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
for imgs, annotations in train_dataloader:
    imgs = list(img.to(device) for img in imgs)
    print(imgs[0].dtype)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)
    break

cpu
torch.float32
[{'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([223]), 'area': tensor([nan])}, {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([162]), 'area': tensor([nan])}, {'boxes': tensor([[272.5639, 237.8576, 320.6771, 285.9708]]), 'labels': tensor([1]), 'image_id': tensor([204]), 'area': tensor([0.0064])}, {'boxes': tensor([[ 83.1192, 220.4503, 146.2872, 283.6183]]), 'labels': tensor([2]), 'image_id': tensor([158]), 'area': tensor([0.0120])}]


In [6]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# ``FasterRCNN`` needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# ``OrderedDict[Tensor]``, and in ``featmap_names`` you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7,
    sampling_ratio=2
)

# put the pieces together inside a Faster-RCNN model
model = FasterRCNN(
    backbone,
    num_classes=2,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)

In [7]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=weights, pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

# 3 features because using classifying into 3 severities: 
# normal, benign, malignant

model = get_model_instance_segmentation(3)

In [8]:
# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2


# get the model using our helper function
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it just for 2 epochs
num_epochs = 2

#num_epochs = config.NUM_EPOCHS
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)

len_dataloader = len(train_dataloader)

train_losses = []  # Store epoch losses
val_loss = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for imgs, annotations in train_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model([imgs[0]], [annotations[0]])
        losses = sum(loss for loss in loss_dict.values())        

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 

        epoch_loss += losses.item()  # Make sure to use .item() to extract scalar from tensor
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    train_losses.append(epoch_loss)

    #model.eval()
    #test_loss = 0

    #for imgs, annotations in test_dataloader:
    #    imgs = list(img.to(device) for img in imgs)
    #    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    #    loss_dict = model([imgs[0]], [annotations[0]])
    #    losses = sum(loss for loss in loss_dict.values()) 
    #    test_loss += losses.item()
    
    #val_loss.append(test_loss)





# Plotting
plt.figure()
plt.plot(range(1, num_epochs+1), train_losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Total Loss')
plt.title('Training Loss per Epoch')
plt.grid(True)
plt.show()


IndexError: Target 2 is out of bounds.

In [ ]:
for imgs, annotations in test_dataloader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        break

In [ ]:
model.eval()
preds = model(imgs)
preds